In [1]:
import numpy as np
import itertools
import functools
import z3
import time
from tqdm.contrib.concurrent import process_map

In [2]:
n = 3
def encode(m):
    return sum(2**k for k, e in enumerate(m.flatten()) if e)

def decode(c, a=n, b=3):
    return np.array([1 if c & (1 << k) else 0 for k in range(a*b)], dtype=object).reshape(a, b)

In [3]:
F = GF(2)

In [4]:
def tp(*ms):
    if len(ms) == 1:
        return ms[0]
    return np.tensordot(ms[0], tp(*ms[1:]), axes=0)

In [5]:
def t2p(t):
    return list(t.flatten())

In [6]:
def get_t(n):
    T = np.zeros((n,)*6, dtype=object)
    for i, j, k in itertools.product(range(n), repeat=3):
        T[i, j, j, k, k, i] = 1
    return T

In [7]:
E = np.eye(n, dtype=object)
T = get_t(n) - tp(E, E, E)

In [8]:
def fill(t):
    m = t2p(t).count('*')
    R = PolynomialRing(F, 'x', m)
    k = 0
    for alpha in itertools.product(*map(range, t.shape)):
        if t[alpha] == '*':
            t[alpha] = R.gen(k)
            k += 1
    return R, t

In [9]:
import pickle
with open(f'result{n}.pickle', 'rb') as fin:
    codes = pickle.load(fin)
codes[:10]

[(238, 239, 247, 91, 91, 219),
 (238, 239, 247, 91, 109, 219),
 (238, 239, 247, 91, 203, 219),
 (238, 239, 247, 91, 211, 219),
 (238, 239, 247, 91, 217, 219),
 (238, 239, 247, 91, 218, 219),
 (238, 239, 247, 109, 109, 219),
 (238, 239, 247, 109, 203, 219),
 (238, 239, 247, 109, 217, 219),
 (238, 239, 247, 109, 218, 219)]

In [10]:
r = 6
R = PolynomialRing(F, 'x', r*3*n*n)
X = np.array(R.gens()).reshape(r, 3, n, n)
Y = sum(tp(u, v, w) + tp(v, w, u) + tp(w, u, v) for u, v, w in X)
polys = list({R(p) for p in t2p(Y - T)} - {0})
print(len(polys))
s = z3.Solver()
dct = {f'x{i}' : z3.Bool(f'x{i}') for i in range(R.ngens())} | {'1' : True}
for p in polys:
    s.add(z3.Not(functools.reduce(z3.Xor, (z3.And(*(dct[a] for a in q.split('*'))) for q in str(p).split(' + ')))))

249


In [11]:
def process_one(code):
    s.push()
    for i in range(r):
        m = decode(code[i])
        for j in range(3):
            for k in range(n):
                v = dct[str(X[i, j, k, k])]
                s.add(v if m[k, j] else z3.Not(v))
    if s.check().r != -1:
        print(code)
        with open(f'success{n}', 'a') as fsuc:
            print(code, file=fsuc)
    s.pop()

In [ ]:
%%time
process_map(process_one, codes, max_workers=4, chunksize=16);